# **Titanic 데이터를 이용한 <span style="color:darkgreen">머신러닝</span> 문제**
---

> **<span style="color:red">다음 문항을 풀기 전에 </span>아래 코드를 실행하시오.**<br>
> 반드시 코드와 주석을 읽고 문제를 푸시오. <br>
> 반드시 출력된 데이터 설명을 읽고 문제를 푸시오.

---



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

x, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

print( x.shape, y.shape)
print(y[:4])
x.head()

(1309, 13) (1309,)
0    1
1    1
2    0
3    0
Name: survived, dtype: category
Categories (2, object): ['0', '1']


,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"


# **Q1. x에서 다음의 컬럼을 삭제하시오.**
---------------------------------
**삭제할 컬럼들**
1. name
2. ticket
2. boat
3. body
4. home.dest
---------------------------

In [2]:
####################
## Your code here ##
####################
x = x.drop(columns=['name', 'ticket', 'boat','body', 'home.dest'])
x.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
0,1.0,female,29.0000,0.0,0.0,211.3375,B5,S
1,1.0,male,0.9167,1.0,2.0,151.5500,C22 C26,S
2,1.0,female,2.0000,1.0,2.0,151.5500,C22 C26,S
3,1.0,male,30.0000,1.0,2.0,151.5500,C22 C26,S
4,1.0,female,25.0000,1.0,2.0,151.5500,C22 C26,S


# Q2. x의 cabin 컬럼을 다음 조건에 맞추어 변형하시오.
-------------------
* 값의 맨 앞글자만 남긴다.
    * 예시) C22 C26 -> C
    * 예시) B5 -> B
* 값이 빈값일 경우, 'N'을 남긴다.
--------------------

In [3]:
x['cabin'].unique

<bound method Series.unique of 0            B5
1       C22 C26
2       C22 C26
3       C22 C26
4       C22 C26
         ...   
1304       None
1305       None
1306       None
1307       None
1308       None
Name: cabin, Length: 1309, dtype: object>

In [4]:
def getFIrstString(x) :
    if x == None:
        return 'N'
    return x[0]

In [5]:
####################
## Your code here ##
###################



x['cabin']=x['cabin'].apply(getFIrstString)


In [6]:
x.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
0,1.0,female,29.0000,0.0,0.0,211.3375,B,S
1,1.0,male,0.9167,1.0,2.0,151.5500,C,S
2,1.0,female,2.0000,1.0,2.0,151.5500,C,S
3,1.0,male,30.0000,1.0,2.0,151.5500,C,S
4,1.0,female,25.0000,1.0,2.0,151.5500,C,S


# Q3. 다음 조건에 맞추어 x에 새로운 컬럼 fs를 만드시오.
-------------------
* sibsp와 parch의 합을 기준으로
    - 1이하 : 'a' 를 값으로 갖는다.
    - 1초과 4이하 : 'b' 를 값으로 갖는다.
    - 4초과 : 'c' 를 값으로 갖는다.
----------------

In [7]:
####################
## Your code here ##
####################
x['fs'] = x['sibsp'] + x['parch']

In [8]:
type(x['fs'][0])

numpy.float64

In [9]:
def convertFS(x) :
    if float(x) <= 1.0:
        return 'a'
    elif float(x)>1.0 and float(x)<=4.0:
        return 'b'
    return 'c'
    


In [10]:
x['fs'] = x['fs'].apply(convertFS)

In [11]:
x.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,fs
0,1.0,female,29.0000,0.0,0.0,211.3375,B,S,a
1,1.0,male,0.9167,1.0,2.0,151.5500,C,S,b
2,1.0,female,2.0000,1.0,2.0,151.5500,C,S,b
3,1.0,male,30.0000,1.0,2.0,151.5500,C,S,b
4,1.0,female,25.0000,1.0,2.0,151.5500,C,S,b


# Q4. x와 y를 트레이닝/벨리데이션/테스트 셋으로 분리하시오.
----------------------
* 변수명 규칙 :
    - x_train, x_valid, x_test
    - y_train, y_valid, y_test
* train : valid : test = 6 : 2 : 2
* random state, seed 등은 2021로 고정
------------------------

In [12]:
####################
## Your code here ##
####################
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x,y, train_size=0.6)
x_valid, x_test, y_valid, y_test = train_test_split(x_valid,y_valid, test_size=0.5)

print(x_train.shape), print(y_train.shape)
print(x_valid.shape), print(y_valid.shape)
print(x_test.shape), print(y_test.shape)

(785, 9)
(785,)
(262, 9)
(262,)
(262, 9)
(262,)


(None, None)

# Q5. 카테고리 데이터들에 빈값이 있다면, 최빈값을 찾아 imputing하시오.
----------------------------
* 모든 전처리 규칙은 트레이닝 셋을 바탕으로 찾는다.
* 대상 컬럼 : pclass, cabin, embarked
* 모든 x들에 대해 진행
----------------------------

In [13]:
####################
## Your code here ##
####################

x['pclass'].fillna(x['pclass'].mode()[0], inplace = True)
x['cabin'].fillna(x['cabin'].mode()[0], inplace = True)
x['embarked'].fillna(x['embarked'].mode()[0], inplace = True)

print('plcass describe', x['pclass'].describe())
print('cabin describe', x['cabin'].describe())
print('embarked describe', x['embarked'].describe())



plcass describe count    1309.000000
mean        2.294882
std         0.837836
min         1.000000
25%         2.000000
50%         3.000000
75%         3.000000
max         3.000000
Name: pclass, dtype: float64
cabin describe count     1309
unique       9
top          N
freq      1014
Name: cabin, dtype: object
embarked describe count     1309
unique       3
top          S
freq       916
Name: embarked, dtype: object


# Q6. 연속형 데이터들에 빈값이 있다면, 중앙값을 찾아 imputing하시오.
----------------------------
* 모든 전처리 규칙은 트레이닝 셋을 바탕으로 찾는다.
* 대상 컬럼 : age, fare
* 모든 x들에 대해 진행
----------------------------

In [14]:
####################
## Your code here ##
####################

x['age'].fillna(x['age'].median(), inplace = True)
x['fare'].fillna(x['fare'].median(), inplace = True)

print('age describe', x['age'].describe())
print('fare describe', x['fare'].describe())

age describe count    1309.000000
mean       29.503183
std        12.905246
min         0.166700
25%        22.000000
50%        28.000000
75%        35.000000
max        80.000000
Name: age, dtype: float64
fare describe count    1309.000000
mean       33.281086
std        51.741500
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: fare, dtype: float64


# Q7. 카테고리 데이터들을 dummy 변수화 하시오.
-------------------------------
* 모든 전처리 규칙은 트레이닝셋을 바탕으로 찾는다.
* 대상 컬럼 : pclass, cabin, embarked, fs, sex
* 더미변수를 만든 후, 원본 변수는 삭제한다.
* 모든 x들에 대해 진행
* 카테고리 안에 4개 클래스가 있다면, 3개 컬럼이 만들어져야 한다.
* x_valid와 x_test의 컬럼들은 x_train의 컬럼과 종류와 순서가 같아야 한다.
-------------------------------

In [15]:
####################
## Your code here ##
####################
target_col = ['pclass', 'cabin','embarked', 'fs', 'sex']
x_train = pd.get_dummies(x_train, columns = target_col)
x_test = pd.get_dummies(x_test, columns = target_col)
x_valid = pd.get_dummies(x_valid, columns = target_col)


print("x_train", x_train.head())
print("x_test", x_test.head())
print("x_valid", x_valid.head())



x_train       age  sibsp  parch     fare  pclass_1.0  pclass_2.0  pclass_3.0  cabin_A  \
866   2.0    0.0    1.0  12.2875           0           0           1        0   
622   4.0    4.0    2.0  31.2750           0           0           1        0   
933  26.0    2.0    0.0   8.6625           0           0           1        0   
494  24.0    1.0    1.0  37.0042           0           1           0        0   
680   NaN    0.0    0.0   7.2250           0           0           1        0   

     cabin_B  cabin_C  ...  cabin_G  cabin_N  embarked_C  embarked_Q  \
866        0        0  ...        0        1           0           0   
622        0        0  ...        0        1           0           0   
933        0        0  ...        0        1           0           0   
494        0        0  ...        0        1           1           0   
680        0        0  ...        0        1           1           0   

     embarked_S  fs_a  fs_b  fs_c  sex_female  sex_male  
866           

# Q8. 다음 조건에 맞추어 의사결정나무들을 학습시키시오.
-----------------------
* max depth는 3으로 고정한다.
* min samples leaf 1개짜리 나무부터 1개씩 늘려서 min samples leaf가 30개인 나무까지, 총 30개의 의사결정 나무를 학습시킨다.
* 학습된 의사결정나무들은 trees 변수에 리스트로 담아둔다.
-----------------------

In [16]:
x_train.describe()

,age,sibsp,parch,fare,pclass_1.0,pclass_2.0,pclass_3.0,cabin_A,cabin_B,cabin_C,...,cabin_G,cabin_N,embarked_C,embarked_Q,embarked_S,fs_a,fs_b,fs_c,sex_female,sex_male
count,627.000000,785.000000,785.000000,784.000000,785.000000,785.000000,785.000000,785.000000,785.000000,785.000000,...,785.000000,785.000000,785.000000,785.000000,785.000000,785.000000,785.000000,785.000000,785.000000,785.000000
mean,29.887826,0.509554,0.384713,33.090832,0.256051,0.205096,0.538854,0.019108,0.048408,0.077707,...,0.002548,0.782166,0.200000,0.112102,0.685350,0.787261,0.165605,0.047134,0.352866,0.647134
std,14.780316,1.054738,0.904789,48.274392,0.436729,0.404029,0.498806,0.136993,0.214763,0.267881,...,0.050443,0.413037,0.400255,0.315693,0.464672,0.409505,0.371963,0.212060,0.478166,0.478166
min,0.166700,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,28.000000,0.000000,0.000000,14.477100,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,39.000000,1.000000,0.000000,31.275000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
max,80.000000,8.000000,9.000000,512.329200,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
y_train.describe()

count     785
unique      2
top         0
freq      487
Name: survived, dtype: object

In [19]:
####################
## Your code here ##
####################
from sklearn.ensemble import RandomForestClassifier

trees = []
for i in range(1, 31):
    rf = RandomForestClassifier(min_samples_leaf = i, max_depth=3)
    rf.fit(x_train, y_train)
    trees.append(rf)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

# Q9. 학습된 의사결정나무들의 성능을 벨리데이션셋에서 시각화 하시오.
-------------------
* 벨리데이션 셋에서 accuracy를 계산하여 시각화 한다.
* x축은 min sample leaf의 수, y축은 accuracy
* line chart를 그린다.
* grid line을 추가한다.
--------------------

In [ ]:
####################
## Your code here ##
####################

accs = []
for tree in trees:
    value = tree.score(x_vlalid, y_valid)
    accs.append(value)
    
    

In [ ]:
plt.plot(list(range(1,31), accs)
plt.show()

# Q10. Q9를 참고하여 가장 성능 좋은 (동일 성능이라면 가장 min sample leaf가 큰) 나무를 골라 테스트셋 위에서 평가하시오.
----------------
1. accuracy를 주석으로 명시하시오.
2. 1을 positive로 두었을 때, sensitivity를 주석으로 명시하시오.
3. 1을 positive로 두었을 때, specificity를 주석으로 명시하시오.

In [ ]:
####################
## Your code here ##
####################

